# Algoritmo APriori Implementacion propia

Estudiante:Edgar Daniel Ramos ALvarez

Cod:171570


## 1.-Instalar bibliotecas

In [94]:
import numpy as np  
import pandas as pd  
from itertools import combinations

## 2.-Importar datos


In [160]:
#Importamos los dato
Sp_data =np.load('spotify.npy',allow_pickle=True)
#Transformamos a un iterable
X=Sp_data.tolist()
Spotify_List=[X[k] for k in X]
#tranformamos en un dataframe
Spotify_df=pd.DataFrame(Spotify_List)

## 3.- Preprocesar (lista de listas) ⏮ 

In [151]:
Spotify_df.shape

(10000, 250)

In [161]:
#Se uso este tamaño donde se puedo encontrar reglas de asociacion y no saturar la memoria de mi computador
Spotify_df=Spotify_df.iloc[0:150,0:100]

In [162]:
Spotify_df.shape

(150, 100)

In [115]:
Spotify_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,Lose Control (feat. Ciara & Fat Man Scoop),Toxic,Crazy In Love,Rock Your Body,It Wasn't Me,Yeah!,My Boo,Buttons,Say My Name,Hey Ya! - Radio Mix / Club Mix,...,Beep,Somebody To Love,Dirty Little Secret,Baby,A Thousand Miles,Livin on Sunday,See You Again,How Do You Sleep? - Featuring Ludacris,This Is Me,My Happy Ending
1,Fathoms Below - Broadway Cast Recording,Daughters Of Triton - Broadway Cast Recording,The World Above - Broadway Cast Recording,Human Stuff - Broadway Cast Recording,I Want the Good Times Back - Broadway Cast Rec...,Part Of Your World,Storm at Sea - Broadway Cast Recording,Part Of Your World (Reprise) - Broadway Cast R...,She's in Love - Broadway Cast Recording,Her Voice - Broadway Cast Recording,...,Overture,Arabian Nights,One Jump Ahead,One Jump Ahead (Reprise),Proud of Your Boy,These Palace Walls,"Babkak, Omar, Aladdin, Kassim",A Million Miles Away,Diamond in the Rough,Friend Like Me
2,ABC,Point/Counterpoint,Billie Jean,Basket Case,Seven Nation Army,1985,Rollup - Baauer Remix,Dirt Off Your Shoulder,"So Ruff, So Tuff",Snow (Hey Oh) - Hey Oh Album Version,...,Cold As Ice - Single / LP Version,Don't Stop Believin',Somebody To Love - Remastered 2011,Bohemian Rhapsody - Remastered 2011,We Own the Night (feat. Luciana) [Original Mix],Eyes On Fire - Zeds Dead Remix,Take Over Control - Radio Edit,Renegade,Sweet Emotion,Play That Funky Music
3,What Is Love 2016 - Mike Mago Remix,Love Me Now - Dave Audé Remix Radio Edit,Thinkin of U,I Will Be Here - (Wolfgang Gartner Remix),The Party (This Is How We Do It),I'll Take You - Solidisco Remix,Show Me Love (Radio Edit),Strike It Up,Hold Me Up - AM2PM Remix,Should’ve Been Us - DubVision Remix,...,None,None,None,None,None,None,None,None,None,None
4,Circles,Ramble Tamble,Brendan 1,Death of a Ladies' Man,After Laughter (Comes Tears),"How You Doin, Here We Go","Please, Please, Please - Live At The Apollo Th...",Wild Witch,Gut Feeling,Good Morning Judge,...,Road To Nowhere - 2005 Remastered Version,Sun Sinking Low,Chaos Engine,Planet of the Dreamers,Summertime,The Big Ship - 2004 Digital Remaster,Come and Get Your Love,Wintersong,You Don't Love Me,Same Thing All Over
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,I Can't Hear You,Blue Blood Blues,60 Feet Tall,I Cut Like A Buffalo,So Far From Your Weapon,Rocking Horse,Bone House,Blue Blood Blues,New Fang,No One Loves Me & Neither Do I,...,None,None,None,None,None,None,None,None,None,None
96,Fathers' Eyes (Live),Arwen's Vigil,A Thousand Years,The Wanderer,CAN'T STOP THE FEELING! (Original Song from Dr...,I Took A Pill In Ibiza - Seeb Remix,Wanna Be That Song,Beethoven's 5 Secrets (Live),Shut Up and Dance,Sugar,...,September,Modern Love - Single Version;1990 Remastered V...,How Would You Feel (Paean),Stay With Me,Titanium / Pavane,Saltwater Gospel,Only Wanna Be With You,Firework,Oh My My,Pompeii
97,Sneakin’,Gassed Up,You Was Right,Fire Squad - Live,A Tale of 2 Citiez - Live,Deja Vu,20 Joints,Moves,Look At Me!,Rent Money,...,None,None,None,None,None,None,None,None,None,None
98,Over the Rainbow / Simple Gifts,Just the Way You Are,Rockelbel's Canon (Pachelbel Canon in D),All of Me,Charlie Brown Medley,Me and My Cello (Happy Together),Can't Help Falling in Love,Love Story,Viva La Vida,Wake Me Up,...,None,None,None,None,None,None,None,None,None,None


## 4.- Modulos Implementados


### Datos Unitarios
Funcion para obtener los items individuales con un umbral minimo


In [155]:
def DatosUnitarios(Lista,minSupport):
  Items=[]
  for x in Lista:
    Items+=x
  #Almacenamos todas la ocurrencias de los items en general
  OcurrenciasTotales=(list(Items))
  #Eliminamos repetidos
  Items=list(set(Items))
  #Filtramos los elementos que pasen el minimo soporte
  Items=list(filter(lambda x:(OcurrenciasTotales.count(x)>minSupport),Items))
  return Items

### get_frequent_itemsets
*Descripcion* de datos de entrada:
```
# dataset:el dataset de las transacciones  lista de listas de str
# min_support:soporte minimo int
# TotalItems:La funcion devuelve lista de listas de str
```


      
      

In [156]:
def get_frequent_itemsets(dataset, min_support):
  #Lista para almacenar las transacciones
  records = []
  #Eliminar los valores None  
  for i in range(0, len(dataset)):  
    TempList=[]
    for j in range(0, len(dataset.columns)):
      if (dataset.values[i,j]!=None):
        #Se compruba que el valor sea diferente diferente de NaN
        TempList.append(dataset.values[i,j])
      else:
        continue
    records.append(TempList)
  print('len-records',len(records))
  #Preprocesamiento obtenemos todos los items sin repeticiones se uso 2 para evitar un calculos excesivo y saturacion de memoria
  Items=DatosUnitarios(records,2)
  print('len-Items',len(Items))
  #El maximo numero de items por transaccion
  #MaxIter=len(max(records,key=len))
  MaxIter=5
  print('MaxIter',MaxIter)
  #Nuestro lista principal que ira creciendo
  TotalItems=[]
  TotalItems+=Items
  #Iteramos desde 2 por que prepocesamos lo datos individuales
  #hasta el maximo numero de items por transaccion
  #podria usarse un while y condicion hasta q ya no haya combinaciones
  #tambiens seria valido
  for i in range(2,MaxIter+1):
    #realizamos la combinaciones
    Combinaciones=combinations(Items,i)
    Combinaciones=list(map(lambda x:list(x),Combinaciones))
    NewItems=[]
    #adaptamos las tuplas
    for ItemBuscado in Combinaciones:
      TItemB=len(list(filter(lambda x:set(ItemBuscado) <= set(x),records)))
      #Comprobamos que las combinaciones superen el minimo soporte
      if TItemB>=min_support:
        NewItems.append(ItemBuscado)
    TotalItems+=NewItems
    #Actualizamos nuestros items para la siguiente iteracion con combinaciones mayores
    Items=DatosUnitarios(NewItems,0)
  #En este caso un filtro a partir de 2 items
  TotalItems=list(filter(lambda x:type(x)!=str,TotalItems))
  return TotalItems

### generate_association_rules
*Descripcion* de datos de entrada:
```
# frequent_itemsets:Itemset obtenidos de get_frequent_itemsets lista de lista de str
# dataset:el dataset de las transacciones  lista de listas de str
# confidence:confianza minima int
# lift:suntento minimo int
```

In [157]:
def generate_association_rules(frequent_itemsets,dataset,confidence=0,lift=0):
  #Lista para almacenar las transacciones
  records = []
  #Eliminar los valores NAN  
  for i in range(0, len(dataset)):  
    TempList=[]
    for j in range(0, len(dataset.columns)):
      if (dataset.values[i,j]!=None):
        #Se compruba que el valor sea diferente diferente de NaN
        TempList.append(dataset.values[i,j])
      else:
        continue
    records.append(TempList)
  #Declaro mi arreglo de reglas de asociacion
  Asosation_rules=[]
  for itemset in frequent_itemsets:
    #El consecuente sera el ultimo producto de la transaccion
    Consecuente=itemset[-1]
    #El antecedente sera el o los productos anteriores al consecuente
    Antecedente=itemset[:-1]
    #Calculamos la confianza y el lift para poder almacenar la transaccion como 
    #regla de asociacion
    #Calculamos el soporte del antecedente
    SupItem=(len(list(filter(lambda x:set(itemset) <= set(x),records))))/len(records)
    SupAnt=(len(list(filter(lambda x:set(Antecedente) <= set(x),records))))/len(records)
    #Calculamos el soporte del consecuente
    Confianza=SupItem/SupAnt
    #Calculamos el lit
    Lift=Confianza/SupAnt
    #print('-------------------')
    #print(itemset)
    #print(SupAnt)
    #print(Lift)
    if ((Confianza>=confidence)and(Lift>=lift)):
      Asosation_rules.append([itemset,SupItem,Confianza,Lift])
  return Asosation_rules

##5.-Usando el codigo

## Prueba con dataset Spotify


### Obteniendo Itemsets

In [170]:
#Definimos las variables generales
Soporte_minimo=4
Confianza=0.5
lift=1.2

In [171]:
Prueba=get_frequent_itemsets(Spotify_df,Soporte_minimo)
print('Itemsets encontrados:',len(Prueba))

len-records 150
len-Items 394
MaxIter 5
Itemsets encontrados: 14


### Obteniendo reglas de asosiacion

In [172]:
Reglas=generate_association_rules(Prueba,Spotify_df,0.5,1.2)
h=1
print("TODAS LAS REGLAS")
for w in Reglas:
    print("Nro:",h)
    print("Regla: " + ",".join(w[0][:-1]) + " -> " + w[0][-1])
    print("Soporte: " + str(w[1]*100))
    print("Confianza: " + str(w[2]*100))
    print("Sustento: " + str(w[3]))
    print("=====================================")
    h+=1

TODAS LAS REGLAS
Nro: 1
Regla: Chasing Cars -> All I Want
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5
Nro: 2
Regla: Uber Everywhere -> goosebumps
Soporte: 2.666666666666667
Confianza: 80.0
Sustento: 24.0
Nro: 3
Regla: Neighbors -> Deja Vu
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5
Nro: 4
Regla: Bounce Back -> Bad and Boujee (feat. Lil Uzi Vert)
Soporte: 2.666666666666667
Confianza: 57.14285714285714
Sustento: 12.244897959183673
Nro: 5
Regla: Bounce Back -> Low Life
Soporte: 2.666666666666667
Confianza: 57.14285714285714
Sustento: 12.244897959183673
Nro: 6
Regla: Bounce Back -> Black Beatles
Soporte: 2.666666666666667
Confianza: 57.14285714285714
Sustento: 12.244897959183673
Nro: 7
Regla: King Kunta -> m.A.A.d city
Soporte: 2.666666666666667
Confianza: 80.0
Sustento: 24.0
Nro: 8
Regla: Low Life -> Black Beatles
Soporte: 2.666666666666667
Confianza: 66.66666666666667
Sustento: 16.666666666666668
Nro: 9
Regla: It's A Vibe -> 1-800-273-8255
Soporte: 2.6666666

### Filtrando reglas
Los Criterios de calidad seleccionados
Confianza y Sustento los valores con mayor Confianza y sustento seran nuestras reglas mas prometedoras

In [173]:
ReglasFiltradas=sorted(Reglas, key=lambda x: (x[2],x[3]),reverse=True)
ReglasFiltradas=ReglasFiltradas[:10]
h=1
for w in ReglasFiltradas:
    print("REGLAS FILTRADAS")
    print("Nro:",h)
    print("Regla: " + ",".join(w[0][:-1]) + " -> " + w[0][-1])
    print("Soporte: " + str(w[1]*100))
    print("Confianza: " + str(w[2]*100))
    print("Sustento: " + str(w[3]))
    print("=====================================")
    h+=1

REGLAS FILTRADAS
Nro: 1
Regla: Chasing Cars -> All I Want
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5
REGLAS FILTRADAS
Nro: 2
Regla: Neighbors -> Deja Vu
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5
REGLAS FILTRADAS
Nro: 3
Regla: pick up the phone -> Deja Vu
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5
REGLAS FILTRADAS
Nro: 4
Regla: Low Life,Bounce Back -> Black Beatles
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5
REGLAS FILTRADAS
Nro: 5
Regla: Uber Everywhere -> goosebumps
Soporte: 2.666666666666667
Confianza: 80.0
Sustento: 24.0
REGLAS FILTRADAS
Nro: 6
Regla: King Kunta -> m.A.A.d city
Soporte: 2.666666666666667
Confianza: 80.0
Sustento: 24.0
REGLAS FILTRADAS
Nro: 7
Regla: Low Life -> Black Beatles
Soporte: 2.666666666666667
Confianza: 66.66666666666667
Sustento: 16.666666666666668
REGLAS FILTRADAS
Nro: 8
Regla: It's A Vibe -> 1-800-273-8255
Soporte: 2.666666666666667
Confianza: 66.66666666666667
Sustento: 16.666666666666668
RE

### Analisis de reglas
Seleccione las siguientes 4 reglas:

----------------------------------------------------------------------------
Nro: 10
Regla: Bounce Back -> Bad and Boujee (feat. Lil Uzi Vert) 
Soporte: 2.666666666666667
Confianza: 57.14285714285714
Sustento: 12.244897959183673
#### Esta regla fue la que tuve la menor confianza y por ende el menor sustento dentre las 10 filtradas la calidad no es muy buena al solo tener un %2.7 en su soporte esto debido a que se tuve q reducir el dataset por los limites de mi algoritmo implementado dandole un analisis de cualidad las 2 canciones se asemejan al estar en un mismo genero el trap y al tener a artistas muy parecidos por ende se entiende la causalidad

----------------------------------------------------------------------------
Nro: 1
Regla: Chasing Cars -> All I Want
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5

#### Esta regla fue la que tuve la que mejor confianza y sustento presento dentre las 10 filtradas la calidad se matiene y un soporte de  %2.7.respecto a las canciones amabas son pop relajadas en este cancion factor muy similar es que en ambas se usan pianos que descatan a lo largo de toda la cancion.
----------------------------------------------------------------------------
Nro: 7
Regla: Low Life -> Black Beatles
Soporte: 2.666666666666667
Confianza: 66.66666666666667
Sustento: 16.666666666666668

#### Una regla dentro del promedio margenes de confianza y sustento no presenta mucha calidad y el sporte se mantiene en %2.7.respecto a las canciones presentadas en un mismo genero el TRAP y con cantantes con timbres de voz muy similares.

----------------------------------------------------------------------------
Nro: 4
Regla: Low Life,Bounce Back -> Black Beatles
Soporte: 2.666666666666667
Confianza: 100.0
Sustento: 37.5

#### Esta regla es una de las pocas que se compone de 2 canciones como antecedentes con una confianza del %100 y sustento de 37.5 margenes mejores las 3 canciones mantienen el genero TRAP y artistas similares.
